In [1]:
import igraph as ig
import numpy as np
import louvain
import time

print("Construct the graph from the txt file")
input = np.loadtxt("../data/PolBlogs/PolBlogsAdj.txt", dtype='i', delimiter='\t');
dim = input.shape;
N = dim[0];

g = ig.Graph();
g.add_vertices(N);
for i in range(0, N-1):
    for j in range(0,N-1):
        if input[i,j]==1:
            g.add_edges([(i,j)]);
ig.summary(g)

print("Largest connected component")
largest = g.clusters().giant();
ig.summary(largest)

/var/folders/b7/w5j5st2x5d9f77725jtxfcqh0000gn/T/ipykernel_62938/1078182516.py:3: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


Construct the graph from the txt file
IGRAPH U--- 1490 18959 -- 
Largest connected component
IGRAPH U--- 1221 18958 -- 


/var/folders/b7/w5j5st2x5d9f77725jtxfcqh0000gn/T/ipykernel_62938/1078182516.py:20: DeprecationWarning: Graph.clusters() is deprecated; use Graph.connected_components() instead
  largest = g.clusters().giant();


In [2]:
print("Running Louvain 1 times in C++")
start_time = time.time()

part_best = louvain.find_partition(largest, louvain.ModularityVertexPartition);

        
print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part_best.modularity)
print("Sizes of the groups:", part_best.sizes())

Running Louvain 1 times in C++
--- 0.11590909957885742 seconds ---
Modularity : 0.43204641398016086
Sizes of the groups: [642, 512, 41, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2]


In [3]:
print("Running Louvain 1000 times in C++")
start_time = time.time()

m_best =0;
part_best = louvain.find_partition(largest, louvain.ModularityVertexPartition);


for n in range(1000):
    part = louvain.find_partition(largest, louvain.ModularityVertexPartition);
    m = part.modularity;
    if m_best<m:
        m_best = m;
        part_best = part;
        

print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part_best.modularity)
print("Sizes of the groups:", part_best.sizes())

Running Louvain 1000 times in C++
--- 119.01615786552429 seconds ---
Modularity : 0.43232728454516384
Sizes of the groups: [626, 522, 45, 5, 4, 3, 3, 3, 2, 2, 2, 2, 2]


In [4]:
from node2vec import Node2Vec


/Users/mfanuel/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import networkx as nx
import numpy as np
from node2vec import Node2Vec

largest_nx = largest.to_networkx()

# Precompute probabilities and generate walks 
node2vec = Node2Vec(largest_nx, dimensions=64, walk_length=30, num_walks=200, workers=1)  # Use temp_folder for big graphs

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)  

Generating walks (CPU: 1):  86%|████████▌ | 171/200 [03:27<00:26,  1.10it/s]

KeyboardInterrupt: 